In [ ]:
# Install required packages
!pip install torch torchvision torchaudio torch-geometric
!pip install networkx matplotlib

In [ ]:
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
print("Number of nodes:", data.num_nodes)
print("Number of edges:", data.num_edges)
print("Number of features:", data.num_node_features)
print("Number of classes:", dataset.num_classes)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx
G = to_networkx(data, to_undirected=True)
plt.figure(figsize=(10, 8))
nx.draw_spring(G, node_size=10, alpha=0.7)
plt.title("Cora Citation Network")
plt.show()

In [ ]:
class GCN(torch.nn.Module):
 def __init__(self):
  super().__init__()
  self.conv1 = GCNConv(dataset.num_node_features, 16)
  self.conv2 = GCNConv(16, dataset.num_classes)
 def forward(self, x, edge_index):
  x = self.conv1(x, edge_index)
  x = F.relu(x)
  x = F.dropout(x, training=self.training)
  x = self.conv2(x, edge_index)
  return F.log_softmax(x, dim=1)
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [ ]:
def train():
  model.train()
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  loss.backward()
  optimizer.step()
  return loss.item()
for epoch in range(1, 201):
  loss = train()
  if epoch % 20 == 0:
    print(f'Epoch {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy on test set: {acc:.4f}')